In [99]:
# Import Packages

import tensorflow as tf
import numpy as np
import pandas as pd
import re
# For Data Preparation
import nltk.data
nltk.download('punkt')
tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')

# For Word Embedding
import gensim
from gensim.models import Word2Vec
from gensim.models import Phrases
import logging

# For LSTM Model
from tensorflow.contrib import rnn
import pprint


from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical



pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()
tf.set_random_seed(777)  # reproducibility

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/eunbeejang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/Users/eunbeejang/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


# Data Preparation

In [100]:
# Data Processing

'''
#Read File
with open('FILE NAME', 'r') as f:
    bias_data = f.read()
'''

# Names for the columns (Header)
cols = ['sentence', 'bias', 'sex', 'age', 'occupation', 'citizenship']

# Load the data from a CSV (returns type(dataframe))
bias_data = pd.read_csv('./scrapeBIG.csv', sep=',',  encoding='latin-1') # encoding='latin-1', names=cols, header=None,
print("Data loaded")


# Randomize data
bias_data = bias_data.reindex(np.random.permutation(bias_data.index))

# Replace NAN with Zero(0)
bias_data.fillna(0, inplace=True)


"""
# Accessing the data
bias_data.value
bias_data[rows:cols]

# Basic Stats about the data
bias_data.describe()
bias_data.count
"""

Data loaded


'\n# Accessing the data\nbias_data.value\nbias_data[rows:cols]\n\n# Basic Stats about the data\nbias_data.describe()\nbias_data.count\n'

In [101]:



bias_data[:10]




,Sentence,Bias,Sex,Age,Occupation,Citizenship
74,The case against TCS was filed in 2015 by Stev...,0.0,0.0,0.0,0.0,0.0
110,The program aims to attract researchers from a...,0.0,0.0,0.0,0.0,0.0
487,"Says program head David Barrett: ""So many of o...",0.0,0.0,0.0,0.0,0.0
489,"""Eventually,"" he says, ""men will start going b...",0.0,0.0,0.0,0.0,0.0
394,I found that these menÕs relationships with ot...,0.0,0.0,0.0,0.0,0.0
378,Even when women and men behave in leaderly way...,0.0,0.0,0.0,0.0,0.0
694,I felt no pressure and I wasn't trying to prov...,0.0,0.0,0.0,0.0,0.0
246,If you shared the biological genitals of a wom...,0.0,0.0,0.0,0.0,0.0
264,That's not to say that there aren't times when...,0.0,0.0,0.0,0.0,0.0
123,The question I wanted to answer was: Did revie...,0.0,0.0,0.0,0.0,0.0


In [102]:
sentence = bias_data['Sentence']
label = bias_data['Bias']

In [103]:
#print(sentence)
#print(label)
print(sentence[0])

Imagine walking down the street and seeing advertising screens change their content based on how you walk, how you talk, or even the shape of your chest.


In [104]:
# Convert a sentence into a list of words

def sentence_to_wordlist(sentence, remove_stopwords=False):
    # 1. Remove non-letters
    sentence_text = re.sub(r'[^\w\s]','', sentence)
    # 2. Convert words to lower case and split them
    words = sentence_text.lower().split()
    # 3. Return a list of words
    return(words)

In [105]:
sentence_to_wordlist(sentence[0])

['imagine',
 'walking',
 'down',
 'the',
 'street',
 'and',
 'seeing',
 'advertising',
 'screens',
 'change',
 'their',
 'content',
 'based',
 'on',
 'how',
 'you',
 'walk',
 'how',
 'you',
 'talk',
 'or',
 'even',
 'the',
 'shape',
 'of',
 'your',
 'chest']

In [108]:
# List of sentences where each sentence is a list of words
def data_to_sentences(data, tokenizer, remove_stopwords=False ):
    try:
        # 1. Use the NLTK tokenizer to split the text into sentences
        raw_sentences = tokenizer.tokenize(data.strip())
        # 2. Loop over each sentence
        sentences = []
        for raw_sentence in raw_sentences:
            # If a sentence is empty, skip it
            if len(raw_sentence) > 0:
                # Otherwise, call sentence_to_wordlist to get a list of words
                sentences.append(sentence_to_wordlist(raw_sentence))
        # 3. Return the list of sentences (each sentence is a list of words, so this returns a list of lists)
        len(sentences)
        #print('GOOD')
        return sentences
    except:
        print('ERROR :', data)

In [116]:
#bias_sentences = bias_data['sentence'].tolist()

sentences = []

for i in range(0,len(sentence)):
    #try:
        # Need to first change "./." to "." so that sentences parse correctly
    data = sentence[i] #.replace("/.", '')
        # Now apply functions
    sentences += data_to_sentences(data, tokenizer)
    #except:
    #    print('Nonono! ERROR in ... creating a lit of sentences' ')

print("There are " + str(len(sentences)) + " sentences in the corpus of bias_data.")
              
# Accessing each sentences
              
# sentences[index]

There are 895 sentences in the corpus of bias_data.


In [110]:
#labels = bias_data['bias'].tolist()

# Word Embedding

In [115]:
# Create Word Vectors

# train model
model = Word2Vec(sentences, min_count=1)

# Calling init_sims will make the model will be better for memory if we don't want to train the model over and over again
model.init_sims(replace=True)

# Summarize the loaded model
print(model)

# Summarize vocabulary
words = list(model.wv.vocab)
#print(words)

print(model['case'])
print(model['program'])
"""

# save model
model.save('model.bin')

# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

# access vector for one word
print(model['sentence'])


"""


Word2Vec(vocab=3825, size=100, alpha=0.025)
[-0.10636672 -0.14265263 -0.02078844 -0.0232397  -0.11916558 -0.01144145
 -0.17669465 -0.04775187 -0.02393734  0.08522997  0.03947792  0.04068775
  0.14863844 -0.04276646  0.056711   -0.00946584 -0.07331948  0.01676305
  0.22920586  0.11169035 -0.09725992  0.05375832  0.02720733  0.06562116
 -0.08984692 -0.14993925  0.03146018 -0.05324413 -0.22514331 -0.10860506
 -0.04654963 -0.15635628  0.28395778 -0.04567298 -0.07397024  0.00344704
  0.18910787 -0.07491151 -0.01727845 -0.00496828  0.02231907  0.06447957
  0.08648467 -0.0159786  -0.07405483  0.00616721  0.0440251  -0.19083989
  0.0012248  -0.05227284 -0.17266096  0.12702642 -0.01785207  0.10279123
 -0.02607189 -0.05348418 -0.03739962 -0.04781531 -0.10755043 -0.14032726
  0.05204801 -0.14130357 -0.02954207 -0.11977896 -0.01943543  0.05340746
  0.10656927  0.06546345  0.08271471 -0.02562047  0.00571597  0.02433201
  0.19804275 -0.1523862  -0.04308036  0.00227266 -0.07295103  0.15554589
  0.092

/Users/eunbeejang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
/Users/eunbeejang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


"\n\n# save model\nmodel.save('model.bin')\n\n# load model\nnew_model = Word2Vec.load('model.bin')\nprint(new_model)\n\n# access vector for one word\nprint(model['sentence'])\n\n\n"

# LSTM Model Test (tensor flow)

In [2]:
# RNN/LSTM Model

# One cell RNN input_dim(4) -> output_dim(2)
# sequnce_length 
num_classes = 5
learning_rate = 0.1
hidden_size = 5 # size of output from the LSTM
input_dim = 5 # one hot size
sequence_length = 6 # |hihello| == 6
batch_size = 1 # num of words to be inputted each training



# Data Creation
# idx2char: dictionary
idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3
y_data = [[1, 0, 2, 3, 3, 4]]    # ihello

"""
# One hot Encoding
h = [1, 0, 0, 0, 0] # 0:h
i = [0, 1, 0, 0, 0] # 1:i
e = [0, 0, 1, 0, 0] # 2:e
l = [0, 0, 0, 1, 0] # 3:l
o = [0, 0, 0, 0, 1] # 4:0

# shape of x_data : (batch size, sequence length, one hot vocab vector size)
x_data = np.array([[h,e,l,l,o],[e,o,l,l,l],[l,l,e,e,l]], dtype=np.float32)
print("x_data.shape = ", x_data.shape)
pp.pprint(x_data)
print("\n")


cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True, reuse=tf.AUTO_REUSE)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
sess.run(tf.global_variables_initializer())

# shape of output : (batch size, sequence length, hidden size)
print("output.shape = ", outputs.shape)
pp.pprint(outputs.eval())
print("\n")

"""


X = tf.placeholder(tf.float32, [None, sequence_length, input_dim]) # X one-hot, None: Batch size
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label


# Basic LSTM
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True, reuse=tf.AUTO_REUSE)
initial_state = cell.zero_state(batch_size, tf.float32) # Initial State is always ZERO
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)


# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
# fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# fc_b = tf.get_variable("fc_b", [num_classes])
# outputs = tf.matmul(X_for_fc, fc_w) + fc_b
outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)



# Calculate Sequence_Loss
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)

train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)


# Teach RNN 'ihello'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "\nprediction: ", result, "\ntrue Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\nPrediction str: ", ''.join(result_str), "\n==============")










0 loss: 1.60788 
prediction:  [[3 3 3 3 3 3]] 
true Y:  [[1, 0, 2, 3, 3, 4]]

Prediction str:  llllll 
1 loss: 1.51026 
prediction:  [[3 3 3 3 3 3]] 
true Y:  [[1, 0, 2, 3, 3, 4]]

Prediction str:  llllll 
2 loss: 1.4327 
prediction:  [[3 3 3 3 3 3]] 
true Y:  [[1, 0, 2, 3, 3, 4]]

Prediction str:  llllll 
3 loss: 1.34895 
prediction:  [[3 3 3 3 3 3]] 
true Y:  [[1, 0, 2, 3, 3, 4]]

Prediction str:  llllll 
4 loss: 1.25513 
prediction:  [[1 3 3 3 3 3]] 
true Y:  [[1, 0, 2, 3, 3, 4]]

Prediction str:  illlll 
5 loss: 1.14044 
prediction:  [[1 3 3 3 3 3]] 
true Y:  [[1, 0, 2, 3, 3, 4]]

Prediction str:  illlll 
6 loss: 1.01676 
prediction:  [[1 3 2 3 3 4]] 
true Y:  [[1, 0, 2, 3, 3, 4]]

Prediction str:  ilello 
7 loss: 0.896927 
prediction:  [[1 3 2 3 3 4]] 
true Y:  [[1, 0, 2, 3, 3, 4]]

Prediction str:  ilello 
8 loss: 0.769525 
prediction:  [[1 0 2 3 3 4]] 
true Y:  [[1, 0, 2, 3, 3, 4]]

Prediction str:  ihello 
9 loss: 0.655007 
prediction:  [[1 0 2 3 3 4]] 
true Y:  [[1, 0, 2, 3, 3

# LSTM MODEL for Bias Analysis